In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

# Check for CUDA device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader with shuffle enabled
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.relu(self.conv3(x))
        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # No softmax since CrossEntropyLoss includes it


# Initialize model, loss, and optimizer
learning_rate = 0.003
L2 = 0.0001
beta = 0.99

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, alpha=beta, weight_decay=L2)

# Training loop
epochs = 30
total_start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    epoch_start_time = time.time()
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    epoch_end_time = time.time()
    
    # Evaluate model on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}, "
          f"Test Accuracy: {test_accuracy:.2f}%, Time: {epoch_end_time - epoch_start_time:.2f} sec")

total_end_time = time.time()
print(f"Total Training Time: {total_end_time - total_start_time:.2f} sec")


Epoch 1/30, Loss: 0.2269, Test Accuracy: 98.17%, Time: 6.68 sec
Epoch 2/30, Loss: 0.0577, Test Accuracy: 98.84%, Time: 6.50 sec
Epoch 3/30, Loss: 0.0412, Test Accuracy: 98.61%, Time: 6.43 sec
Epoch 4/30, Loss: 0.0307, Test Accuracy: 98.84%, Time: 6.44 sec
Epoch 5/30, Loss: 0.0255, Test Accuracy: 99.16%, Time: 6.33 sec
Epoch 6/30, Loss: 0.0208, Test Accuracy: 99.25%, Time: 6.36 sec
Epoch 7/30, Loss: 0.0181, Test Accuracy: 99.21%, Time: 6.49 sec
Epoch 8/30, Loss: 0.0156, Test Accuracy: 99.16%, Time: 6.42 sec


KeyboardInterrupt: 